In [1]:
import pandas as pd
import numpy as np
import gmplot
from Turning import check_turn  # , quality_turn

In [27]:
# data = pd.read_csv('Swift//MarL2.csv')
data = pd.read_csv('data//14-Aug-2021-4.csv')
# print(data[' Bearing'])
df = pd.DataFrame()
df['time'] = data['Trip Time(Since journey start)(s)']
df['Latitude'] = data[' Latitude']
df['Longitude'] = data[' Longitude']
df['Compass angle'] = data[' Bearing'].astype('float')
df['Speed'] = data['Speed (OBD)(km/h)'].astype('float')

df['Compass angle'] = df['Compass angle'].replace(0, np.nan)
df['Compass angle'] = df['Compass angle'].replace(method='ffill')
# df = df.dropna(axis=0)
df = df[df['time']>0]
print(df.shape)
# df.set_index('time', inplace=True)

(620, 5)


In [28]:
Speed = df['Speed']
Angle = df['Compass angle']
len(Angle)

620

In [29]:
spot1 = []
spot2 = []
i = 0
while i < (df.shape[0] - 10):
    if abs(Angle.iloc[i] - Angle.iloc[i + 9]) >= 60:
        spot1.append(i)
        spot2.append(i + 9)
        # ans.append(check_turn(i, i + 10))
        i = i + 10
    else:
        i += 1

print(spot1)
print(spot2)

[30, 42, 165, 178, 204, 217, 263, 273, 384, 420, 480, 517, 552, 563, 586, 596, 606]
[39, 51, 174, 187, 213, 226, 272, 282, 393, 429, 489, 526, 561, 572, 595, 605, 615]


In [21]:
x = np.array(Speed.iloc[139:148+1])
n = len(x)
n

10

In [33]:
def quality_turn2(speed, start, end):
    x = np.array(speed.iloc[start:end+1])
    n = len(x)
    
    score = 0
    for i in range(n):
        # if speed.iloc[start+i] - speed.iloc[start + i + 1] >= 0:
        if speed.iloc[start+i] >= speed.iloc[start + i + 1]:
            score += 1
        else:
            score += 0
    # return n, score, ((score*100)/(n-1))
    return round((score / (n+2)) * 100, 2)

In [23]:
print(quality_turn2(Speed, 36, 45))
print(quality_turn2(Speed, 47, 56))

100.0
66.67


In [24]:
def quality_turn3(speed, start, end):
    x = np.array(speed.iloc[start:end+1])
    n = len(x)
    Velocity = []
    # score = 0
    for i in range(n):
        Velocity.append(speed.iloc[start+i])
    print(Velocity)

quality_turn3(df['Speed'], 154, 163)

[20.0, 23.0, 23.0, 26.0, 26.0, 29.0, 29.0, 29.0, 29.0, 29.0]


In [53]:
Lati = []
Longi = []
for i in range(len(spot1)):
    T = check_turn(Angle, spot1[i], spot2[i])
    print(T)
    y = quality_turn2(Speed, spot1[i], spot2[i])
    if y >= 60:
        print('Good Turn, score is', y, '%')
    else:
        print('Bad Turn, score is', y, '%')
        for j in range(spot2[i] - spot1[i]):
            Lati.append(df['Latitude'].iloc[spot1[i]+j])
            Longi.append(df['Longitude'].iloc[spot1[i]+j])            

Left
Bad Turn, score is 50.0 %
Left
Good Turn, score is 66.67 %
Right
Good Turn, score is 66.67 %
Right
Bad Turn, score is 50.0 %
Left
Good Turn, score is 83.33 %
Left
Bad Turn, score is 58.33 %
Right
Good Turn, score is 66.67 %
Right
Good Turn, score is 66.67 %
Right
Good Turn, score is 66.67 %
Right
Good Turn, score is 83.33 %
Left
Good Turn, score is 83.33 %
Right
Bad Turn, score is 58.33 %
Left
Good Turn, score is 75.0 %
Right
Good Turn, score is 66.67 %
Right
Good Turn, score is 75.0 %
Right
Good Turn, score is 66.67 %
Left
Good Turn, score is 83.33 %


In [54]:
print(Lati)
print(Longi)

[18.62956022, 18.62957969, 18.62960487, 18.62963444, 18.6296605, 18.62968815, 18.62972004, 18.62975324, 18.62979218, 18.62743281, 18.62745096, 18.62746766, 18.62748874, 18.62751544, 18.627539499999997, 18.62755874, 18.62757989, 18.62760723, 18.62895251, 18.62897987, 18.6290076, 18.62903043, 18.62905216, 18.62907395, 18.62909273, 18.62910978, 18.62912476, 18.63010775, 18.6301204, 18.63014252, 18.63015089, 18.63014627, 18.63014318, 18.63013148, 18.63011454, 18.63009879]
[73.78850902, 73.78851004, 73.78851369, 73.78851920000001, 73.78852109, 73.7885216, 73.78852422, 73.78852718, 73.78852454, 73.78218774, 73.78218301, 73.78217948, 73.78217551, 73.78217144, 73.78216819, 73.78216509, 73.78216221, 73.78216021, 73.782487, 73.78247781, 73.78246794, 73.78244922, 73.78242396, 73.78239407, 73.78236061, 73.78232508, 73.78228714, 73.78820553, 73.78823794, 73.7882869, 73.78832215, 73.78834957, 73.78837971, 73.78840796, 73.78843083, 73.78844895]


In [55]:
gmap = gmplot.GoogleMapPlotter(df['Latitude'].iloc[1], df['Longitude'].iloc[1], 17)
gmap.scatter(Lati, Longi, 'red', size=2, marker=False)
gmap.draw("Bad_turns_1.html")

In [38]:
"""def PlotMap(Latitude, Longitude):
    gmap = gmplot.GoogleMapPlotter(df['Latitude'].iloc[1], df['Longitude'].iloc[1], 17)
    gmap.scatter(Latitude, Longitude, 'red', size=2, marker=False)
    gmap.draw("Bad_turns_1.html")"""

In [39]:
PlotMap(Lati, Longi)

In [46]:
n = (spot2[i-3] - spot1[i])
n

-34